In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import os

import tensorflow as tf
import keras
from tensorflow.keras.layers import Permute
from tensorflow.keras import optimizers
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, cohen_kappa_score

from Dataset import config
from Dataset.EEGDataset_SOTA import EEGDataset
from Dataset.misc import subject_run_map
from utils.analyzer import stats
from utils.wrapper_SOTA import ModelWrapper
from utils.callbacks import SimpMovAve
from utils.seed import set_seeds
from utils.gpu import gpu_allocation
from utils.log import Logging, dict_pad, save_checkpoint, load_checkpoint

from SOTA.SOTA_Model import SOTA_Models
from SOTA.preprocessing import standardize_data

In [ ]:
GPU_ALLOCATION = None # None | GPU Memory Limit in GB

gpu_allocation(GPU_ALLOCATION) if GPU_ALLOCATION else None

In [ ]:
ONE_HOT = True # Bool
DATASET_PATH = 'RESOURCES'

EEG_DATASET_CONFIG = {
    'normalization': True, # None | 'batch' | 'channel' | 'time' | 'time-batch'

    'signal_duration': None, # None for Default Dataset tmin & tmax config | [tmin, tmax]
}

In [ ]:
DATASET = 'BCICIV2A'        # 'BCICIV2A' | 'EEGMMIDB'

SUBJECT_SELECTION = 'SL'    # 'SL' | 'SM' | 'SN'
SESSION_SELECTION = 'DS'    # 'DS' | 'RS'
FINE_TUNING = False         # for 'SL-DS-FT' using 'SN' Top Models as baseline

VARY_VALID_RUN = True       # True for DS: LOSeO
KFOLD = False               # True for SN: LSSO | False for SN: LOSO 

CLASS = 4                   # (int)

In [ ]:
if DATASET == 'BCICIV2A':
    SUBJECTS = [subj for subj in range(1, 10)]
    VALID_RUN = [[True],
                 [False]] # [False, True]

    EVENTS = ['Left Hand', 'Right Hand', 'Feet', 'Tongue']
    SUBJECT_EXEMPTION = config.dataset_BCICIV2A.subject_exemption
    DATALOADER_KWARGS = {}

elif DATASET == 'EEGMMIDB':
    SUBJECTS = [subj for subj in range(1, 110)]
    VALID_RUN = [[4, 6],
                 [8, 10],
                 [12, 14]]  # [4, 6, 8, 10, 12, 14]

    SUBJECT_EXEMPTION = config.dataset_EEGMMIDB.subject_exemption
    EVENTS = ['Left Fist', 'Right Fist', 'Both Fists', 'Both Feet']
    DATALOADER_KWARGS = {'Epochs_proj': False}
    
SUBJECT_SIZE = len(SUBJECTS)
SUBJECT_SIZE = 1 if (SUBJECT_SELECTION == 'SM') else SUBJECT_SIZE
SUBJECT_SIZE = 5 if (SUBJECT_SELECTION == 'SN' and KFOLD) else SUBJECT_SIZE # 20% train-test split

EVENTS = EVENTS[:CLASS]

In [ ]:
def load_dataset(DATASET, SUBJECT_SELECTION, SESSION_SELECTION, SUBJECT, VALID_RUN, SEED=42):

    (TRAIN_SUBJECT, TRAIN_RUN), (VALID_SUBJECT, VALID_RUN) = subject_run_map(DATASET, SUBJECT_SELECTION, SESSION_SELECTION, SUBJECT, VALID_RUN)

    X, y = EEGDataset(DATASET, {i: TRAIN_RUN for i in TRAIN_SUBJECT}, EVENTS,
                      EEG_DATASET_CONFIG,
                      one_hot=ONE_HOT, dataset_path=DATASET_PATH,
                      **DATALOADER_KWARGS)

    if SESSION_SELECTION == 'RS':
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)

        del X, y
        
    else: # 'DS'
        X_train, y_train = X, y
        X_test, y_test = EEGDataset(DATASET, {i: VALID_RUN for i in VALID_SUBJECT}, EVENTS,
                                    EEG_DATASET_CONFIG,
                                    one_hot=ONE_HOT, dataset_path=DATASET_PATH,
                                    **DATALOADER_KWARGS)

    # X: (S, N, T, 1)
    if EEG_DATASET_CONFIG['normalization']:
        X_train, X_test = standardize_data(X_train, X_test, axis=-3) # wrt CH axis
        
    return X_train, y_train, X_test, y_test

In [ ]:
MODEL_NAME = 'DB-ATCNet'    # 'EEGNet' | 'EEG-TCNet' | 'TCNet-Fusion' | 'ATCNet' | 'DB-ATCNet' 

class EEG_Model(ModelWrapper):
    def __init__(self, classes):
        super().__init__(classes)
        
        self.eeg = SOTA_Models(MODEL_NAME)

    def call(self, input):

        out = Permute((3,1,2))(input)
        # (1, N, T)
        out = self.eeg(out)

        return out

In [ ]:
NUM_TRAIN = 10
LOAD_CHECKPOINT = True

KFOLD_SPLIT_SIZE = 5

EPOCHS = 1000

MOV_WINDOW_SIZE = 20
MOV_AVE = True
MOV_STD = True

# Hyperparameters
BATCH_SIZE = 64

LOSS = 'CCE' if CLASS > 2 else 'BCE' # 'BCE' | 'CCE' | 'Loge'

OPTIMIZER = 'Adam' # 'Adam' | 'AdamW'
if not FINE_TUNING:
    LR = 0.0009 
else:
    LR = 0.0009
WEIGHT_DECAY = None
AMSGRAD = False

# LR_SCHED_METRIC = 'val_loss' # 'val_loss' | 'mov_ave_val_loss'
# LR_SCHED_DECAY = 0.9
# LR_SCHED_MIN = 0.0001
# LR_SCHED_PATIENCE = 10
# LR_SCHED_COOLDOWN = 0

TARGET_METRIC = 'val_accuracy' # 'val_accuracy' | 'mov_ave_val_accuracy'
EARLY_STOP_PATIENCE = 300

In [ ]:
# %%capture

TRAINING_CASE = SUBJECT_SELECTION  if SUBJECT_SELECTION == 'SN' else SUBJECT_SELECTION + '-' + SESSION_SELECTION
TRAINING_CASE = TRAINING_CASE + '-FT' if FINE_TUNING else TRAINING_CASE
TRAINING_CASE = DATASET +'-'+ TRAINING_CASE
results_path = os.path.join(os.getcwd(), '.results', MODEL_NAME, TRAINING_CASE)
if not os.path.exists(results_path):
    os.makedirs(results_path)
checkpoint_path = results_path + '/checkpoint.json'

log_write = Logging(results_path + '/log.txt')

if LOAD_CHECKPOINT and os.path.exists(checkpoint_path):
    # LOAD Last Train Config
    SUBJ_INIT, SUBJ_TRAIN_INIT, SUBJ_SUMMARY, OVERALL_SUMMARY = load_checkpoint(file_path=checkpoint_path)

    # Load Next Train Config
    if SUBJ_SUMMARY:
        SUBJ_INIT += 1
        SUBJ_TRAIN_INIT = 1
    else:
        SUBJ_TRAIN_INIT += 1

    # Load Checkpoint Data
    data = np.load(results_path + '/model_performance.npz')
    
    train_seed = data['train_seed']
    inference_time = data['inference_time']

    test_acc = data['test_acc']
    test_kappa = data['test_kappa']
    test_loss = data['test_loss']
    
    min_test_loss = data['min_test_loss']
    
    max_test_mov_ave_acc = data['max_test_mov_ave_acc']
    min_test_mov_ave_loss = data['min_test_mov_ave_loss']

    best_runs = data['best_runs']
    best_test_acc = data['best_test_acc']
    best_test_kappa = data['best_test_kappa']
    best_test_loss = data['best_test_loss']
    best_min_test_loss = data['best_min_test_loss']

    best_mov_ave_runs = data['best_mov_ave_runs']
    best_max_test_mov_ave_acc = data['best_max_test_mov_ave_acc']
    best_min_test_mov_ave_loss = data['best_min_test_mov_ave_loss']

    log_write.write('\n\n-----LOAD CHECKPOINT-----\n')

    CHECKPOINT_LOADED = True

else:
    # NO CHECKPOINT
    SUBJ_INIT = 1
    SUBJ_TRAIN_INIT = 1
    OVERALL_SUMMARY = False

    log_write.write(MODEL_NAME + '\t' + TRAINING_CASE + '\n')

    # Initialize Data
    train_seed = np.zeros((SUBJECT_SIZE, NUM_TRAIN))
    inference_time = np.zeros((SUBJECT_SIZE, NUM_TRAIN))

    test_acc = np.zeros((SUBJECT_SIZE, NUM_TRAIN))
    test_kappa = np.zeros((SUBJECT_SIZE, NUM_TRAIN))
    test_loss = np.zeros((SUBJECT_SIZE, NUM_TRAIN))

    min_test_loss = np.zeros((SUBJECT_SIZE, NUM_TRAIN))

    max_test_mov_ave_acc = np.zeros((SUBJECT_SIZE, NUM_TRAIN))
    min_test_mov_ave_loss = np.zeros((SUBJECT_SIZE, NUM_TRAIN))

    best_runs = np.zeros(SUBJECT_SIZE, dtype=np.int8)
    best_test_acc = np.zeros(SUBJECT_SIZE)
    best_test_kappa = np.zeros(SUBJECT_SIZE)
    best_test_loss = np.zeros(SUBJECT_SIZE)
    best_min_test_loss = np.zeros(SUBJECT_SIZE)

    best_mov_ave_runs = np.zeros(SUBJECT_SIZE, dtype=np.int8)
    best_max_test_mov_ave_acc = np.zeros(SUBJECT_SIZE)
    best_min_test_mov_ave_loss = np.zeros(SUBJECT_SIZE)

    CHECKPOINT_LOADED = False

if KFOLD:
    kf = KFold(n_splits=KFOLD_SPLIT_SIZE, shuffle=False)
    SUBJECT_KFOLD_SPLIT = [kf_split[1]+1 for kf_split in list(kf.split(SUBJECTS))]

for subj in range(SUBJ_INIT-1, SUBJECT_SIZE):
    log_write.write('\nTraining Subject {:d}\n'.format(subj+1))

    if subj+1 in SUBJECT_EXEMPTION and (SUBJECT_SELECTION == 'SL' or (SUBJECT_SELECTION == 'SN' and not KFOLD)):
        train_seed[subj] = np.full(NUM_TRAIN, np.nan)
        inference_time[subj] = np.full(NUM_TRAIN, np.nan)

        test_acc[subj] = np.full(NUM_TRAIN, np.nan)
        test_kappa[subj] = np.full(NUM_TRAIN, np.nan)
        test_loss[subj] = np.full(NUM_TRAIN, np.nan)

        min_test_loss[subj] = np.full(NUM_TRAIN, np.nan)

        max_test_mov_ave_acc[subj] = np.full(NUM_TRAIN, np.nan)
        min_test_mov_ave_loss[subj] = np.full(NUM_TRAIN, np.nan)

        best_runs[subj] = 0
        best_test_acc[subj] = np.nan
        best_test_kappa[subj] = np.nan
        best_test_loss[subj] = np.nan
        best_min_test_loss[subj] = np.nan

        best_mov_ave_runs[subj] = 0
        best_max_test_mov_ave_acc[subj] = np.nan
        best_min_test_mov_ave_loss[subj] = np.nan
        
        log_write.write('SKIPPED\n')

        with open(results_path + '/model_performance.npz', 'wb') as model_performance:
            np.savez(model_performance,
                    train_seed=train_seed, inference_time=inference_time, 
                    test_acc=test_acc, test_kappa=test_kappa, test_loss=test_loss, min_test_loss=min_test_loss,
                    max_test_mov_ave_acc=max_test_mov_ave_acc, min_test_mov_ave_loss=min_test_mov_ave_loss, 
                    best_runs=best_runs, best_test_acc=best_test_acc, best_test_kappa=best_test_kappa, best_test_loss=best_test_loss, best_min_test_loss=best_min_test_loss,
                    best_mov_ave_runs=best_mov_ave_runs, best_max_test_mov_ave_acc=best_max_test_mov_ave_acc, best_min_test_mov_ave_loss=best_min_test_mov_ave_loss)
        print('Model Performance Saved')

        save_checkpoint(subj+1, NUM_TRAIN, subject_summary=True, file_path=checkpoint_path)
        print('-----CHECKPOINT-----')

        continue

    subject_list = SUBJECT_KFOLD_SPLIT[subj] if (SUBJECT_SELECTION in ['SN'] and KFOLD) else [subj+1]

    if SESSION_SELECTION == 'DS' and not VARY_VALID_RUN:
        X_train, y_train, X_test, y_test = load_dataset(DATASET, SUBJECT_SELECTION, SESSION_SELECTION, subject_list, VALID_RUN[0])
        print('Subject {:d} Dataset Loaded'.format(subj+1))
        print(X_train.shape)
        print(X_test.shape)

    subj_path = results_path + '/subj-{:d}'.format(subj+1)
    if not os.path.exists(subj_path):
        os.makedirs(subj_path)

    for train_run in range(SUBJ_TRAIN_INIT-1, NUM_TRAIN):
        train_path = subj_path + '/run-{:d}'.format(train_run+1)
        best_model_path = train_path + '-best_model.h5'
        train_history_path = train_path + '-train_history.csv'
        train_plot_path = train_path + '-train_plot.png'
        confusion_matrix_path = train_path + '-confusion_matrix.png'
        classification_report_path = train_path + '-classification_report.csv'

        if VARY_VALID_RUN:
            VARY_VALID_RUN_STEP = NUM_TRAIN // len(VALID_RUN)
            if ((train_run % VARY_VALID_RUN_STEP) == 0 and (train_run//VARY_VALID_RUN_STEP) < len(VALID_RUN)) or CHECKPOINT_LOADED:
                CHECKPOINT_LOADED = False
                valid_run_index = train_run//VARY_VALID_RUN_STEP
                X_train, y_train, X_test, y_test = load_dataset(DATASET, SUBJECT_SELECTION, SESSION_SELECTION, subject_list, VALID_RUN[valid_run_index])
                print('Subject {:d} Dataset {:d} Loaded'.format(subj+1, valid_run_index+1))
                print(X_train.shape)
                print(X_test.shape)
        
        SEED = np.random.randint(1e4)
        set_seeds(SEED)
        train_seed[subj, train_run] = SEED
        log_write.write('Subject: {:d}\tTrain No.: {:d}\tSeed: {:d}'.format(subj+1, train_run+1, SEED))

        if SESSION_SELECTION == 'RS':
            X_train, y_train, X_test, y_test = load_dataset(DATASET, SUBJECT_SELECTION, SESSION_SELECTION, subject_list, None, SEED=SEED)
            print('Subject {:d} Dataset Loaded'.format(subj+1))
            print(X_train.shape)
            print(X_test.shape)

        model = EEG_Model(EVENTS)
        model.build_model(X_train[0].shape).summary()
        print('Model Built')

        if FINE_TUNING:
            model(X_train[:2])

            tuned_model_SUBJ = subj+1 if not KFOLD else np.argmax([subj+1 in kf_split for kf_split in SUBJECT_KFOLD_SPLIT])+1

            tuned_model_TRAINING_CASE = DATASET +'-'+ 'SN'
            tuned_model_TRAINING_CASE = tuned_model_TRAINING_CASE + '-KFold' if KFOLD else tuned_model_TRAINING_CASE
            tuned_model_results_path = os.path.join(os.getcwd(), '.results', MODEL_NAME, tuned_model_TRAINING_CASE)
            
            if VARY_VALID_RUN:
                data = np.load(tuned_model_results_path + '/model_performance.npz')
                tuned_model_acc = data['max_test_mov_ave_acc']
                tuned_model_acc = tuned_model_acc[tuned_model_SUBJ-1]
                tuned_model_best_runs = np.argsort(tuned_model_acc)[::-1] + 1

                VARY_VALID_RUN_STEP = NUM_TRAIN // len(VALID_RUN)
                tuned_model_best_run_index = train_run%VARY_VALID_RUN_STEP
                tuned_model_path = tuned_model_results_path + '/subj-{:d}/run-{:d}-best_model.h5'.format(tuned_model_SUBJ, tuned_model_best_runs[tuned_model_best_run_index])

            else:
                tuned_model_path = tuned_model_results_path + '/subj-{:d}/run-{:d}-best_model.h5'.format(tuned_model_SUBJ, train_run+1)
            
            model.load_weights(tuned_model_path)

        model.compile(loss={'BCE': BinaryCrossentropy(), 'CCE': CategoricalCrossentropy()}[LOSS],
                      optimizer=getattr(optimizers, OPTIMIZER, getattr(optimizers.experimental, OPTIMIZER, None))(learning_rate=LR, weight_decay=WEIGHT_DECAY, amsgrad=AMSGRAD),
                      metrics=['accuracy'])
        print('Model Compiled')

        callbacks = [
            ModelCheckpoint(best_model_path, monitor=TARGET_METRIC, mode='max' if 'accuracy' in TARGET_METRIC else 'min' if 'loss' in TARGET_METRIC else None, 
                            save_best_only=True, save_weights_only=True, verbose=0),

            # ReduceLROnPlateau(monitor=LR_SCHED_METRIC, mode='max' if 'accuracy' in LR_SCHED_METRIC else 'min' if 'loss' in LR_SCHED_METRIC else None,
            #                   factor=LR_SCHED_DECAY, min_lr=LR_SCHED_MIN,
            #                   patience=LR_SCHED_PATIENCE, cooldown=LR_SCHED_COOLDOWN, verbose=1),
            EarlyStopping(monitor=TARGET_METRIC, mode='max' if 'accuracy' in TARGET_METRIC else 'min' if 'loss' in TARGET_METRIC else None, 
                          patience=EARLY_STOP_PATIENCE, verbose=1),
                          
            SimpMovAve('accuracy', MOV_WINDOW_SIZE, mov_ave=MOV_AVE, mov_std=MOV_STD), SimpMovAve('loss', MOV_WINDOW_SIZE, mov_ave=MOV_AVE, mov_std=MOV_STD),
            ]

        history = model.fit(X_train, y_train,
                            validation_data=(X_test, y_test),
                            batch_size=BATCH_SIZE,
                            epochs=EPOCHS, verbose=1, callbacks=callbacks)
        
        model.plot(mov_ave=MOV_AVE, mov_std=False, grid_on=True).savefig(train_plot_path)
        print('Train Plot Saved')

        pd.DataFrame(dict_pad(history.history, pad_value=None, pad_pos='leading')).to_csv(train_history_path, index=True)
        print('Train History Saved')

        model.load_weights(best_model_path)
        print('Best Model Loaded')

        y_test_round = np.argmax(y_test, axis=1)

        in_run = time.time()
        test_preds = model.predict(X_test)
        out_run = time.time()
        test_preds = np.argmax(test_preds, axis=1)
        inference_time[subj, train_run] = (out_run - in_run) / len(y_test) * 1000
        log_write.write('\tInference Time: {:.4f} ms'.format(inference_time[subj, train_run]))

        test_acc[subj, train_run] = accuracy_score(y_test_round, test_preds)
        test_kappa[subj, train_run] = cohen_kappa_score(y_test_round, test_preds)
        test_loss[subj, train_run] = model.test(x=X_test, y=y_test)[0]
        min_test_loss[subj, train_run] = min(history.history['val_loss'])

        max_test_mov_ave_acc[subj, train_run] = max(history.history['mov_ave_val_accuracy'])
        min_test_mov_ave_loss[subj, train_run] = min(history.history['mov_ave_val_loss'])
        
        log_write.write('\tTest Accuracy: {:.4f}\tTest Kappa: {:.4f}\tTest Loss: {:.4f}\tMIN Test Loss: {:.4f}'.format(
            test_acc[subj, train_run], test_kappa[subj, train_run], test_loss[subj, train_run], min_test_loss[subj, train_run]))
        log_write.write('\tMAX Test MovAve Accuracy: {:.4f}\tMIN Test MovAve Loss: {:.4f}\n'.format(
            max_test_mov_ave_acc[subj, train_run], min_test_mov_ave_loss[subj, train_run]))
        
        confusion_matrix = model.confusion_matrix((X_test, y_test))
        confusion_matrix.savefig(confusion_matrix_path, bbox_inches='tight')
        plt.show()
        print('Confusion Matrix Saved')
        
        clsf_report = model.classification_report((X_test, y_test))
        print(clsf_report)
        pd.DataFrame(clsf_report).to_csv(classification_report_path, index=True)
        print('Classification Report Saved')

        with open(results_path + '/model_performance.npz', 'wb') as model_performance:
            np.savez(model_performance,
                    train_seed=train_seed, inference_time=inference_time, 
                    test_acc=test_acc, test_kappa=test_kappa, test_loss=test_loss, min_test_loss=min_test_loss,
                    max_test_mov_ave_acc=max_test_mov_ave_acc, min_test_mov_ave_loss=min_test_mov_ave_loss, 
                    best_runs=best_runs, best_test_acc=best_test_acc, best_test_kappa=best_test_kappa, best_test_loss=best_test_loss, best_min_test_loss=best_min_test_loss,
                    best_mov_ave_runs=best_mov_ave_runs, best_max_test_mov_ave_acc=best_max_test_mov_ave_acc, best_min_test_mov_ave_loss=best_min_test_mov_ave_loss)
        print('Model Performance Saved')

        save_checkpoint(subj+1, train_run+1, file_path=checkpoint_path)
        print('-----CHECKPOINT-----')

        plt.close('all')
    
    # SUBJECT SUMMARY
    SUBJ_TRAIN_INIT = 1

    best_runs[subj] = np.argmax(test_acc[subj,:])
    best_test_acc[subj] = test_acc[subj, best_runs[subj]]
    best_test_kappa[subj] = test_kappa[subj, best_runs[subj]]
    best_test_loss[subj] = test_loss[subj, best_runs[subj]]
    best_min_test_loss[subj] = min_test_loss[subj, best_runs[subj]]

    best_mov_ave_runs[subj] = np.argmax(max_test_mov_ave_acc[subj,:])
    best_max_test_mov_ave_acc[subj] = max_test_mov_ave_acc[subj, best_mov_ave_runs[subj]]
    best_min_test_mov_ave_loss[subj] = min_test_mov_ave_loss[subj, best_mov_ave_runs[subj]]

    log_write.write('Subject: {:d}'.format(subj+1))
    log_write.write('\tBEST Run: {:d}\t\t\tBEST Test Accuracy: {:.4f}\tBEST Test Kappa: {:.4f}\tBEST Test Loss: {:.4f}\tBEST MIN Test Loss: {:.4f}\n'.format(
        best_runs[subj]+1, best_test_acc[subj], best_test_kappa[subj], best_test_loss[subj], best_min_test_loss[subj]))
    log_write.write('\tBEST MovAve Run: {:d}\t\t\t\t\t\t\tBEST MAX Test MovAve Accuracy: {:.4f}\tBEST MIN Test MovAve Loss: {:.4f}\n'.format(
        best_mov_ave_runs[subj]+1, best_max_test_mov_ave_acc[subj], best_min_test_mov_ave_loss[subj]))
    log_write.write('\t\t\tAVERAGE Inference Time: {:s} ms'.format(stats(inference_time[subj])))
    log_write.write('\tAVERAGE Test Accuracy: {:s}\tAVERAGE Test Kappa: {:s}\tAVERAGE Test Loss: {:s}\tAVERAGE MIN Test Loss: {:s}'.format(
        stats(test_acc[subj]), stats(test_kappa[subj]), stats(test_loss[subj]), stats(min_test_loss[subj])))
    log_write.write('\tAVERAGE MAX Test MovAve Accuracy: {:s}\tAVERAGE MIN Test MovAve Loss: {:s}\n'.format(
        stats(max_test_mov_ave_acc[subj]), stats(min_test_mov_ave_loss[subj])))

    with open(results_path + '/model_performance.npz', 'wb') as model_performance:
        np.savez(model_performance,
                train_seed=train_seed, inference_time=inference_time, 
                test_acc=test_acc, test_kappa=test_kappa, test_loss=test_loss, min_test_loss=min_test_loss,
                max_test_mov_ave_acc=max_test_mov_ave_acc, min_test_mov_ave_loss=min_test_mov_ave_loss, 
                best_runs=best_runs, best_test_acc=best_test_acc, best_test_kappa=best_test_kappa, best_test_loss=best_test_loss, best_min_test_loss=best_min_test_loss,
                best_mov_ave_runs=best_mov_ave_runs, best_max_test_mov_ave_acc=best_max_test_mov_ave_acc, best_min_test_mov_ave_loss=best_min_test_mov_ave_loss)
    print('Model Performance Saved')

    save_checkpoint(subj+1, train_run+1, subject_summary=True, file_path=checkpoint_path)
    print('-----CHECKPOINT-----')

# OVERALL SUMMARY
if not OVERALL_SUMMARY:   
    log_write.write('\nSUMMARY\n')
    for subj in range(SUBJECT_SIZE):
        log_write.write('Subject: {:d}'.format(subj+1))
        log_write.write('\tBEST Run: {:d}\tBEST Test Accuracy: {:.4f}\tBEST Test Kappa: {:.4f}\tBEST Test Loss: {:.4f}\tBEST MIN Test Loss: {:.4f}\n'.format(
            best_runs[subj]+1, best_test_acc[subj], best_test_kappa[subj], best_test_loss[subj], best_min_test_loss[subj]))
        log_write.write('\tBEST MovAve Run: {:d}\t\t\t\t\tBEST MAX Test MovAve Accuracy: {:.4f}\tBEST MIN Test MovAve Loss: {:.4f}\n'.format(
            best_mov_ave_runs[subj]+1, best_max_test_mov_ave_acc[subj], best_min_test_mov_ave_loss[subj]))
        log_write.write('\t\t\tAVERAGE Inference Time: {:s} ms'.format(stats(inference_time[subj])))
        log_write.write('\tAVERAGE Test Accuracy: {:s}\tAVERAGE Test Kappa: {:s}\tAVERAGE Test Loss: {:s}\tAVERAGE MIN Test Loss: {:s}'.format(
            stats(test_acc[subj]), stats(test_kappa[subj]), stats(test_loss[subj]), stats(min_test_loss[subj])))
        log_write.write('\tAVERAGE MAX Test MovAve Accuracy: {:s}\tAVERAGE MIN Test MovAve Loss: {:s}\n'.format(
            stats(max_test_mov_ave_acc[subj]), stats(min_test_mov_ave_loss[subj])))

    log_write.write('\nAll Subjects - ALL RUNS\n')
    log_write.write('\t\tMAX Test Accuracy: {:.4f}\tMAX Test Kappa: {:.4f}\tMIN Test Loss: {:.4f}\tMIN MIN Test Loss: {:.4f}'.format(
        np.nanmax(test_acc), np.nanmax(test_kappa), np.nanmin(test_loss), np.nanmin(min_test_loss)))
    log_write.write('\tMAX Test MovAve Accuracy: {:.4f}\tMIN Test MovAve Loss: {:.4f}\n'.format(
        np.nanmax(max_test_mov_ave_acc), np.nanmin(min_test_mov_ave_loss)))
    log_write.write('\t\t\tAVERAGE Inference Time: {:s} ms'.format(stats(inference_time)))
    log_write.write('\tAVERAGE Test Accuracy: {:s}\tAVERAGE Test Kappa: {:s}\tAVERAGE Test Loss: {:s}\tAVERAGE MIN Test Loss: {:s}'.format(
        stats(test_acc), stats(test_kappa), stats(test_loss), stats(min_test_loss)))
    log_write.write('\tAVERAGE MAX Test MovAve Accuracy: {:s}\tAVERAGE MIN Test MovAve Loss: {:s}\n'.format(
        stats(max_test_mov_ave_acc), stats(min_test_mov_ave_loss)))

    log_write.write('\nAll Subjects - BEST RUNS\n')
    log_write.write('\t\tMAX Test Accuracy: {:.4f}\tMAX Test Kappa: {:.4f}\tMIN Test Loss: {:.4f}\tMIN MIN Test Loss: {:.4f}'.format(
        np.nanmax(best_test_acc), np.nanmax(best_test_kappa), np.nanmin(best_test_loss), np.nanmin(best_min_test_loss)))
    log_write.write('\tMAX Test MovAve Accuracy: {:.4f}\tMIN Test MovAve Loss: {:.4f}\n'.format(
        np.nanmax(best_max_test_mov_ave_acc), np.nanmin(best_min_test_mov_ave_loss)))
    log_write.write('\t\tAVERAGE Test Accuracy: {:s}\tAVERAGE Test Kappa: {:s}\tAVERAGE Test Loss: {:s}\tAVERAGE MIN Test Loss: {:s}'.format(
        stats(best_test_acc), stats(best_test_kappa), stats(best_test_loss), stats(best_min_test_loss)))
    log_write.write('\tAVERAGE MAX Test MovAve Accuracy: {:s}\tAVERAGE MIN Test MovAve Loss: {:s}\n'.format(
        stats(best_max_test_mov_ave_acc), stats(best_min_test_mov_ave_loss)))

    save_checkpoint(subj+1, train_run+1, subject_summary=True, overall_summary=True, file_path=checkpoint_path)
    print('-----CHECKPOINT-----')

print('Model Training Complete')